In [ ]:
import pandas as pd
import chardet

In [ ]:
# <Prepare a set of paths of each raw data file>

data_files = []

for i in range(1,10):
  item = f"2022_0{i}.csv"
  data_files.append(item)

for i in range(1,3):
  item = f"2022_1{i}.csv"
  data_files.append(item)

for i in range(1,10):
  item = f"2023_0{i}.csv"
  data_files.append(item)

for i in range(1):
  item = f"2023_1{i}.csv"
  data_files.append(item)


In [ ]:
# <Combining & Filtering>

# prepare a dataframe to store merged dataset
merged_df = pd.DataFrame()

for i in data_files:
  file_path = '/content/drive/MyDrive/하늘/raw data/'+i

  data = pd.read_csv(file_path, encoding='cp949')

  # handle read_csv issue
  column_1 = list(data['발전구분'].index)
  column_2 = list(data['발전구분'].values)
  column_3 = list(data[' 호기'].values)
  column_4 = list(data[' 일자'].values)
  for i in range(2):
    column_name = f' {i+1}시 발전량(MWh)'
    globals()[f'column_{5+i}'] = list(data[column_name].values)
  for i in range(21):
    column_name = f'{i+3}시 발전량(MWh)'
    globals()[f'column_{7+i}'] = list(data[column_name].values)
  column_28 = list(data['24시 발전량(MWh)'].values)
  column_29 = list(data['총량(MW)'].values)
  column_30 = list(data['평균(MW)'].values)
  column_31 = list(data['최대(시간별)'].values)
  column_32 = list(data['최소(시간별)'].values)
  column_33 = list(data['최대'].values)

  column_map = {'발전구분': column_1, '호기': column_2, '일자': column_3, '1시 발전량(MWh)': column_4, '2시 발전량(MWh)': column_5, '3시 발전량(MWh)': column_6,
       '4시 발전량(MWh)': column_7, '5시 발전량(MWh)': column_8, '6시 발전량(MWh)': column_9, '7시 발전량(MWh)': column_10,
       '8시 발전량(MWh)': column_11, '9시 발전량(MWh)': column_12, '10시 발전량(MWh)': column_13, '11시 발전량(MWh)': column_14,
       '12시 발전량(MWh)': column_15, '13시 발전량(MWh)': column_16, '14시 발전량(MWh)': column_17, '15시 발전량(MWh)': column_18,
       '16시 발전량(MWh)': column_19, '17시 발전량(MWh)': column_20, '18시 발전량(MWh)': column_21, '19시 발전량(MWh)': column_22,
       '20시 발전량(MWh)': column_23, '21시 발전량(MWh)': column_24, '22시 발전량(MWh)': column_25, '23시 발전량(MWh)': column_26,
       '24시 발전량(MWh)': column_27, '총량(MW)': column_28, '평균(MW)': column_29, '최대(시간별)': column_30, '최소(시간별)': column_31, '최대': column_32, '최소': column_33}

  df = pd.DataFrame(column_map)

  # filtering - extract only those of photovaltaic power generation
  filtered_data = df[df['발전구분'].str.contains('태양광')]

  # filtering - exclude any rows with 0 of power generation
  filtered_data = filtered_data[filtered_data['평균(MW)'] != 0]

  # merging - aggregate dataframe of each month into a single dataset
  merged_df = pd.concat([merged_df, filtered_data], ignore_index=True)

  print(f"filtered_df: {filtered_data.shape}, merged_df: {merged_df.shape}")

filtered_df: (681, 33), merged_df: (681, 33)
filtered_df: (616, 33), merged_df: (1297, 33)
filtered_df: (679, 33), merged_df: (1976, 33)
filtered_df: (659, 33), merged_df: (2635, 33)
filtered_df: (682, 33), merged_df: (3317, 33)
filtered_df: (643, 33), merged_df: (3960, 33)
filtered_df: (663, 33), merged_df: (4623, 33)
filtered_df: (681, 33), merged_df: (5304, 33)
filtered_df: (648, 33), merged_df: (5952, 33)
filtered_df: (653, 33), merged_df: (6605, 33)
filtered_df: (679, 33), merged_df: (7284, 33)
filtered_df: (672, 33), merged_df: (7956, 33)
filtered_df: (616, 33), merged_df: (8572, 33)
filtered_df: (680, 33), merged_df: (9252, 33)
filtered_df: (593, 33), merged_df: (9845, 33)
filtered_df: (601, 33), merged_df: (10446, 33)
filtered_df: (651, 33), merged_df: (11097, 33)
filtered_df: (676, 33), merged_df: (11773, 33)
filtered_df: (665, 33), merged_df: (12438, 33)
filtered_df: (589, 33), merged_df: (13027, 33)
filtered_df: (593, 33), merged_df: (13620, 33)
삼천포태양광       2474
영흥태양광 #3   

In [ ]:
# <Add new column combining power plant & power generator unit>

data = merged_df

# combine power plant and power generator unit into a single name
data['발전기'] = data['발전구분'] + data['호기'].astype(str)

# drop the original columns for each
data = data.drop(['발전구분', '호기'], axis=1)
data.insert(0, '발전기', data.pop('발전기'))

solar_df = data

In [ ]:
# <Disunite each row into 24 rows, each representing each hour>

# prepare a dataframe to store final dataset
final_df = pd.DataFrame()

for i in range(len(solar_df['발전기'].value_counts())):
  tmp_df = pd.DataFrame()
  globals()[f'df_{i+1}'] = solar_df[solar_df['발전기'] == solar_df['발전기'].value_counts().index[i]]

  df_values = globals()[f'df_{i+1}'][globals()[f'df_{i+1}'].columns[:26]]
  df_other_values = globals()[f'df_{i+1}'][globals()[f'df_{i+1}'].columns[26:]]

  # iterate for each day
  for j in range(len(df_values)):
    df_values = df_values.reset_index(drop=True)
    df_other_values = df_other_values.reset_index(drop=True)
    df = pd.DataFrame(df_values.loc[j]).T
    df_other = pd.DataFrame(df_other_values.loc[j]).T

    df['일자'] = pd.to_datetime(df['일자'])
    base_date = df['일자'].dt.strftime('%Y-%m-%d').iloc[0]

    # prepare a new column
    new_columns = {}

    # iterate for each hour
    for hour in range(1, 25):
        old_column_name = f'{hour}시 발전량(MWh)'
        new_column_name = f'{base_date} {hour - 1:02d}:00:00'
        new_columns[old_column_name] = new_column_name
    df = df.rename(columns=new_columns)
    df = df.drop('일자', axis=1)
    df = df.drop('발전기', axis=1)
    df = df.transpose()

    # change column name
    df.columns = ['발전량(MWh)']

    # add columns to df
    df['총량(MW)'] = df_other['총량(MW)'].values[0]
    df['평균(MW)'] = df_other['평균(MW)'].values[0]
    df['최대(시간별)'] = df_other['최대(시간별)'].values[0]
    df['최소(시간별)'] = df_other['최소(시간별)'].values[0]
    df['최대'] = df_other['최대'].values[0]
    df['최소'] = df_other['최소'].values[0]

    # add df to tmp_df
    tmp_df = tmp_df.append(df)

  # add column '발전기' to tmp_df
  tmp_df['발전기'] = solar_df['발전기'].value_counts().index[i]

  tmp_df.insert(0, '발전기', tmp_df.pop('발전기'))
  final_df = final_df.append(tmp_df)

In [ ]:
# reset index

final_df = final_df.reset_index()
final_df.rename(columns={'index': 'date'}, inplace=True)
final_df

,date,발전기,발전량(MWh),총량(MW),평균(MW),최대(시간별),최소(시간별),최대,최소
0,2022-01-31 00:00:00,탑선태양광 3,0.0,742.800,31.0,742.80,742.800,742.80,0.0
1,2022-01-31 01:00:00,탑선태양광 3,0.0,742.800,31.0,742.80,742.800,742.80,0.0
2,2022-01-31 02:00:00,탑선태양광 3,0.0,742.800,31.0,742.80,742.800,742.80,0.0
3,2022-01-31 03:00:00,탑선태양광 3,0.0,742.800,31.0,742.80,742.800,742.80,0.0
4,2022-01-31 04:00:00,탑선태양광 3,0.0,742.800,31.0,742.80,742.800,742.80,0.0
...,...,...,...,...,...,...,...,...,...
326875,2023-10-01 19:00:00,고흥만 수상태양광 1,0.0,357075.348,14878.0,50698.62,51.156,50698.62,0.0
326876,2023-10-01 20:00:00,고흥만 수상태양광 1,0.0,357075.348,14878.0,50698.62,51.156,50698.62,0.0
326877,2023-10-01 21:00:00,고흥만 수상태양광 1,0.0,357075.348,14878.0,50698.62,51.156,50698.62,0.0
326878,2023-10-01 22:00:00,고흥만 수상태양광 1,0.0,357075.348,14878.0,50698.62,51.156,50698.62,0.0


In [ ]:
# <Extract dataframe into csv file>

final_df.to_csv("power_generation.csv")